In [ ]:
import graph_tool as gt
from graph_tool.all import *
import matplotlib.pyplot as plt
import os
from InferC import *
import graph_tool.collection as gtc
import numpy as np
import pickle as cp

In [ ]:
def vmtoG(VM,M): #projects configuration to graph, VM:vertex map of subgraphs M:motifs
    elist=[]
    for i in range(len(VM)):
        for vmap in VM[i]:
            for e in M[i].edges():
                elist.append([vmap[M[i].vertex_index[e.source()]],vmap[M[i].vertex_index[e.target()]]])
    g=gt.Graph(directed=M[0].is_directed())
    g.add_edge_list(elist)
    return g


In [ ]:
#functions for plotting configurations and motifs
def ColorC(VM,M,G,ps,out): #plot graph with edges colored according to motifs
    cl=[list(plt.get_cmap('rainbow')((i+1)/float(len(M)+1))) for i in range(len(M))]
    ecolor=G.new_edge_property('vector<float>')
    G.edge_properties['color']=ecolor
    wid=G.new_edge_property('float',vals=0.2)
    G.edge_properties['width']=wid
    for i in range(len(M)):
        for vmap in VM[i]:
            for e in M[i].edges():
                E=G.edge(vmap[M[i].vertex_index[e.source()]],vmap[M[i].vertex_index[e.target()]])
                #E=G.edge_index[ee]
                
                if M[i].num_edges()==1:
                    G.ep.width[E]=0.4
                    G.ep.color[E]=(0.1, 0.2, 0.5, 0.3)
                else:
                    G.ep.width[E]=2.5
                    G.ep.color[E]=cl[-i]
    graph_draw(G,vertex_size=5,edge_color=ecolor,edge_pen_width=wid,pos=ps, output_size=(1600,1600),output=out,inline=True)
    return G
def ColorCLCC(VM,M,G,ps,out): #same as ColorC but for the largest connected component 
    cl=[list(plt.get_cmap('hsv')((i+1)/float(len(M)+1))) for i in range(len(M))]
    ecolor=G.new_edge_property('vector<float>')
    G.edge_properties['color']=ecolor
    wid=G.new_edge_property('float',vals=0.2)
    G.edge_properties['width']=wid
    for i in range(len(M)):
        for vmap in VM[i]:
            for e in M[i].edges():
                E=G.edge(vmap[M[i].vertex_index[e.source()]],vmap[M[i].vertex_index[e.target()]])
                #E=G.edge_index[ee]
                
                if M[i].num_edges()==1:
                    G.ep.width[E]=0.4
                    G.ep.color[E]=(0.1, 0.2, 0.5, 0.3)
                else:
                    G.ep.width[E]=0.5
                    G.ep.color[E]=cl[-i]
    G = GraphView(G, vfilt=label_largest_component(G))
    graph_draw(G,vertex_size=2,edge_color=ecolor,edge_pen_width=wid,pos=ps, output_size=(600,600),output=out,inline=True)
    return G
def ColorCW(VM,M,G,ps):
    cl=[list(plt.get_cmap('rainbow')((i+1)/float(len(M)+1))) for i in range(len(M))]
    ecolor=G.new_edge_property('vector<float>')
    G.edge_properties['color']=ecolor
    wid=G.new_edge_property('float',vals=0.2)
    G.edge_properties['width']=wid
    for i in range(len(M)):
        for vmap in VM[i]:
            for e in M[i].edges():
                E=G.edge(vmap[M[i].vertex_index[e.source()]],vmap[M[i].vertex_index[e.target()]])
                #E=G.edge_index[ee]
                
                if M[i].num_edges()==1:
                    G.ep.width[E]=0.4
                    G.ep.color[E]=(0.1, 0.2, 0.5, 0.3)
                else:
                    G.ep.width[E]=1.5
                    G.ep.color[E]=cl[-i]
    graph_draw(G,vertex_size=5,edge_color=ecolor,edge_pen_width=wid,pos=ps,inline=True)
    return G
def ColorCS(VM,M,G,ps,S,out):
    cl=[list(plt.get_cmap('hsv')((i+1)/float(len(M)+1))) for i in range(len(M))]
    ecolor=G.new_edge_property('vector<float>')
    G.edge_properties['color']=ecolor
    wid=G.new_edge_property('float',vals=0.2)
    G.edge_properties['width']=wid
    for i in S:
        for vmap in VM[i]:
            for e in M[i].edges():
                E=G.edge(vmap[M[i].vertex_index[e.source()]],vmap[M[i].vertex_index[e.target()]])
                #E=G.edge_index[ee]
                G.ep.color[E]=cl[-i]
                G.ep.width[E]=1.0
    graph_draw(G,vertex_size=5,edge_color=ecolor,edge_pen_width=wid,pos=ps, output_size=(600,600),output=out)
    return G
def ColorC2(VM,M,G):
    cl=[list(plt.get_cmap('Set3')((i)/float(len(M)))) for i in range(len(M))]
    ecolor=G.new_edge_property('vector<float>')
    G.edge_properties['color']=ecolor
    wid=G.new_edge_property('float',vals=1)
    G.edge_properties['width']=wid
    for i in range(len(VM)):
        for vmap in VM[i]:
            for e in M[i].edges():
                E=G.edge(vmap[M[i].vertex_index[e.source()]],vmap[M[i].vertex_index[e.target()]])
                #E=G.edge_index[ee]
                G.ep.color[E]=cl[-i]
                G.ep.width[E]=4
    dellist=[]
    for v in G.vertices():
        if v.out_degree()==0:
            dellist.append(v)
    for v in reversed(sorted(dellist)):
        G.remove_vertex(v)
    ps=sfdp_layout(G)
    graph_draw(G,vertex_size=5,edge_color=G.ep.color,edge_pen_width=G.ep.width,pos=ps, output_size=(600,800))
    return G
def DrawM(m): #draws m with colors representing orbits
    c=['r','b','g','yellow','purple','grey','black','orange']
    vcolor=m.new_vp('string',vals=[c[i] for i in m.gp.orbmem])
    graph_draw(m,vertex_fill_color=vcolor)
    return 0
def DrawMF(m,outfile): 
    c=['r','b','g','yellow','purple','grey','black','orange']
    vcolor=m.new_vp('string',vals=[c[i] for i in m.gp.orbmem])
    if outfile:
        graph_draw(m,vertex_fill_color=vcolor,output=outfile+'.pdf')
    graph_draw(m,vertex_fill_color=vcolor)
    return 0


In [ ]:
#example
cov='facebook_friendsAU8lad'
th=open(cov,'rb')
C=cp.load(th)
th.close()
VM=C[0]
M=C[1]
g=vmtoG(VM,M)
pos=sfdp_layout(g)
G=ColorCW(VM,M,g,pos)    
print(g)



In [ ]:
#plot motifs and print frequencies 
for i in range(len(M)):
    print(M[i],len(VM[i]))
    DrawM(M[i])
    

In [ ]:
import os
def get_queries_S(indir,s): #get list of files/networks in indir with extention 's'
    queries=[]
    for root, dirs, filenames in os.walk(indir):
        for f in filenames:
            if f[-len(s):]==s: 
                queries.append(os.path.join(root, f))
    queries.sort()
    return queries

In [ ]:
def SigmaCM(g,M): #description length of g according to configuration model
    m0=M[-1].copy()
    me=g.new_gp('object',val=m0)
    g.gp.me=me
    ed=g.new_graph_property("int",val=g.num_edges())
    g.gp.E=ed
    oivp = g.new_vertex_property("int",vals=g.vertex_index.copy())
    g.vp.OI=oivp
    cove=g.new_edge_property("bool",val=True)
    g.ep.covered=cove
    g.set_edge_filter(g.ep.covered)
    vorder = g.new_vertex_property("int")
    g.vp.vorder=vorder
    return SIGMAO_E([],[],g.num_vertices(),g.gp.E,g,g.gp.me)
def SigmaER(g,M):#description length of g according to ER model
    m0=M[-1].copy()
    me=g.new_gp('object',val=m0)
    g.gp.me=me
    ed=g.new_graph_property("int",val=g.num_edges())
    g.gp.E=ed
    oivp = g.new_vertex_property("int",vals=g.vertex_index.copy())
    g.vp.OI=oivp
    cove=g.new_edge_property("bool",val=True)
    g.ep.covered=cove
    g.set_edge_filter(g.ep.covered)
    vorder = g.new_vertex_property("int")
    g.vp.vorder=vorder
    return SIGMAH_E([],[],g.num_vertices(),g.gp.E,g,g.gp.me)
def DLs(filename): #gives DLs for all model variants 
    th=open(filename,'rb')
    C=cp.load(th)
    th.close()
    VM=C[0]
    M=C[1]
    g=vmtoG(VM,M)
    print(g)
    for m in M:
            orbtype=m.new_vertex_property('int')
            m.vp.orbtype=orbtype
            orbtc=m.new_gp("vector<int>",val=[0,0,0])
            m.gp.orbtc=orbtc
            for v in m.vertices():
                if v.out_degree() and v.in_degree():
                    m.gp.orbtc[2]+=1
                    m.vp.orbtype[v]=2
                elif v.out_degree():
                    m.vp.orbtype[v]=0
                    m.gp.orbtc[0]+=1
                elif v.in_degree():
                    m.vp.orbtype[v]=1
                    m.gp.orbtc[1]+=1
    
    print(filename,len(M),'\n H:',SIGMAH(VM,M,g.num_vertices(),g.num_edges()),',\n T:',SIGMAA(VM,M,g.num_vertices(),g.num_edges()),',\n O:',SIGMAO(VM,M,g.num_vertices(),g.num_edges()),',\n M:',SIGMANO(VM,M,g.num_vertices(),g.num_edges()),',\n D:',SIGMADAS(VM,M,g.num_vertices(),g.num_edges()),',\n ConfModel:',SigmaCM(g,M),',\n ER:',SigmaER(g,M))

In [ ]:
#example
DLs('facebook_friendsAU8lad')